### Train word2vec using imdb_review

The codes are based on Yuan's codes that train doc2vec on imdb_review. See manage/pyml4/expt/gensim/doc2vec.py

In [1]:
import pandas as pd

In [2]:
from __future__ import print_function
from os.path import dirname
import os,sys,inspect

current_dir = dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
manage_dir = dirname(dirname(dirname(current_dir)))

if not manage_dir in sys.path:
    sys.path.insert(0, manage_dir)

# from pyml4.common import context, db
# context.local_context.print_summary()

import gensim
from gensim.models import Word2Vec
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import logging
import nltk
import nltk.data
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import SnowballStemmer
from nltk.corpus import webtext
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
from nltk.corpus import stopwords
import pymysql.cursors

In [3]:
logger = logging.getLogger(__name__)
sent_detector = None

# store NLTK resources inside ML4
#nltk_download_dir = os.path.dirname(nltk.__file__)
#nltk.data.path.append(os.path.dirname(nltk.__file__))
#nltk.download('punkt', download_dir=os.path.dirname(nltk.__file__))
nltk.download('punkt')
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')

NUM_REVIEWS_TO_FETCH = 1000000

[nltk_data] Downloading package punkt to /home/evazhong/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [14]:
class ReviewSentences(object):
    # def __init__(self):
    #     self.db_client = db.manually_connect()
    #
    # def __del__(self):
    #     self.db_client.close()

    def __iter__(self):
        connection = pymysql.connect(host="127.0.0.1",
                                    user="web",
                                    password="atth1132",
                                    db="ML3_mirror")

        with connection.cursor() as cur:
        #with self.db_client.cursor() as cur:
        #cur.execute("select imdbReviewId, movieId, body, upVotes, downVotes from imdb_review ")
            cur.execute("""select imdbReviewId, movieId, body, upVotes, downVotes
                            FROM (select imdbReviewId, movieId, body, upVotes, downVotes,
                            @rn:= if(@movieId = movieId, @rn+1, 1) as rn,
                            @movieId:=movieId as pi
                            from imdb_review, (select @rn:=0) b
                            order by movieId, (upVotes-downVotes)desc) x
                            where x.rn <= 80 and x.pi < 5500
                            """)
            for (review_id, movie_id, body, up, down) in cur:
                for sentence in sent_detector.tokenize(body):
                    yield (movie_id, sentence)

In [15]:
review_sentences = [(movie_id,sentence) for (movie_id,sentence) in ReviewSentences()]

In [17]:
#Remove punctuations
#nltk.download('stopwords')
nltk.download('stopwords')
tokenizer = RegexpTokenizer(r'\w+')

stop_words = set(stopwords.words('english'))
print(stop_words)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/evazhong/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
{'ma', 'myself', "doesn't", 'my', 'and', 'haven', 'doing', "it's", 'on', 'shouldn', 'off', "wouldn't", 'most', 'ourselves', 'of', "you'll", 'because', 'here', "hasn't", "don't", 'them', "haven't", 'by', 'aren', 'than', 'both', 'itself', 'y', 'can', 'do', 'be', "she's", 's', 'hers', 'will', 'just', 'should', 'now', 'under', 'wouldn', 'her', 'when', 'to', 'only', "weren't", 'been', 'who', 'you', 'very', "didn't", 'm', 'won', 'over', 'how', 'isn', "you're", 'being', 'that', "that'll", 'has', 'its', 'wasn', "wasn't", 'your', 'an', 'during', 'couldn', 'through', 'up', 'nor', 'their', 'all', 'at', 'which', 'did', 'what', 'with', 'these', "you'd", 'we', 've', "shan't", 'mightn', "you've", 'the', 'from', 'where', 'our', "isn't", 'once', "needn't", 'while', 'between', 'don', 'does', 'few', 'for', 'further', 'about', 'below', "hadn't", "couldn't", 'into', 'no', 

In [18]:
stemmer = SnowballStemmer("english")
lemma = WordNetLemmatizer()

# stem and remove stopwords from reviews
review_sent = []
only_sentence = []
review_word = []
#
nltk.download('wordnet')

for i in review_sentences:
    #print("Hi")
    #print(i)
    word_lst = [lemma.lemmatize(word,pos='v') for word in tokenizer.tokenize(i[1].lower()) if (word not in stop_words)]
    review_word.append((i[0],word_lst))
    sent = ' '.join(word_lst)
    review_sent.append((i[0],sent))
    only_sentence.append(sent)
print('finish lemmatizing')    

[nltk_data] Downloading package wordnet to /home/evazhong/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
finish lemmatizing


In [19]:
sentence_list = []
for movieId, sentence in review_word:
    sentence_list.append(sentence)


In [27]:
# Store the sentences from imdb_review in a csv_file, so it's easier to reuse it.

import csv
with open('imdb_review_sentences.csv', 'w') as myfile:
#     wr = csv.writer(myfile, delimiter=' ', quoting=csv.QUOTE_MINIMAL)
#     wr.writerow(sentence_list)
    
# with open(csvfile, "w") as output:
    writer = csv.writer(myfile, lineterminator='\n')
    for val in sentence_list:
        writer.writerow([val])   

### Train word2vec model

Note that trianing the model can take from 30 mintues to a few hours. 
An example of the model can be found here: https://drive.google.com/drive/folders/1EBqxmAPAVHxgtaz32PUp0xXsF98B3-GP?usp=sharing


In [29]:
print('start training model2')
word_vec_model2 = gensim.models.Word2Vec(sentence_list, alpha=0.025, size=1000, min_alpha=0.025,min_count=10,workers=35,iter=20)
#word_vec_model1.train(only_sentence,total_examples=word_vec_model1.corpus_count,epochs=word_vec_model1.iter)
word_vec_model2.save('Word2Vec_fullds_model2')
print('second model trained')

start training model2
second model trained


In [31]:
len(word_vec_model2.wv.vocab)

65012

In [34]:
word_vec_model2.wv.similar_by_word("happy")

[('happier', 0.27740973234176636),
 ('happily', 0.2672647535800934),
 ('happiest', 0.22986197471618652),
 ('upbeat', 0.22156524658203125),
 ('glad', 0.20897459983825684),
 ('optimistic', 0.20554432272911072),
 ('hopeful', 0.20425133407115936),
 ('unhappy', 0.19973137974739075),
 ('cheerful', 0.19736379384994507),
 ('awww', 0.18560005724430084)]

### Load word2vec model 

In [3]:
word_vec_model2 = Word2Vec.load('Word2Vec_fullds_model2')

In [4]:
len(word_vec_model2.wv.vocab)

65012

In [8]:
word_vec_model2.wv.similar_by_word("end",topn=20)

[('conclusion', 0.29712265729904175),
 ('finale', 0.2874034345149994),
 ('climax', 0.2586871087551117),
 ('denouement', 0.25783979892730713),
 ('resolution', 0.2184992879629135),
 ('dénouement', 0.21513140201568604),
 ('outcome', 0.21137632429599762),
 ('coda', 0.2093307375907898),
 ('scene', 0.2077297866344452),
 ('begin', 0.20636282861232758),
 ('final', 0.19717377424240112),
 ('finalé', 0.19580122828483582),
 ('resolutions', 0.19251397252082825),
 ('conclude', 0.1850592941045761),
 ('closure', 0.18355096876621246),
 ('finish', 0.1759783774614334),
 ('start', 0.1751575767993927),
 ('completion', 0.17112597823143005),
 ('denouements', 0.16981804370880127),
 ('epilogue', 0.16950276494026184)]